#### 异常检测问题总结

- 对数据没有太多业务认识情况下，要找到异常数据，通常就可以使用异常检测算法
- IsolationForest 在异常检测的问题中，使用频率比较高
  - 对数据要求比较低（不需要做离散化，不需要数值标准化，不需要考虑特征间的关系（例如共线性）等，不需要额外做特征过滤和筛选）
- IsolationForest  对异常点做初步的判断，在结果之上进一步分析规则
- 异常检测问题，在线上实时监测都会落到规则上

- 要求：
  - 知道使用的场景 对数据没有太多业务认识情况下，要找到异常数据，通常就可以使用异常检测算法
  - 记住用什么算法  IsolationForest  one-class SVM LOF
  - 记住算法的原理
    - 孤立森林算法通过对样本点的孤立来检测异常值
      - 该算法利用一种名为孤立树iTree的二叉搜索树结构来孤立样本
      - 异常值的数量较少且与大部分样本的特征不同
      - 异常值会被更早的孤立出来，也即异常值会距离![iTree](https://math.jianshu.com/math?formula=iTree)的根节点更近，而正常值则会距离根节点有更远的距离
      - 相较于LOF，K-means等传统算法，孤立森林算法对高纬数据有较好的鲁棒性。
    - 假设数据集有N条数据，构建一颗iTree时，从N条数据中均匀抽样(一般是无放回抽样)出ψ个样本出来，作为这颗树的训练样本。在样本中，随机选一个特征，并在这个特征的所有值范围内(最小值与最大值之间)随机选一个值，对样本进行二叉划分，将样本中小于该值的划分到节点的左边，大于等于该值的划分到节点的右边。由此得到一个分裂条件和左、右两边的数据集，然后分别在左右两边的数据集上重复上面的过程，直到数据集只有一条记录或者达到了树的限定高度

####  对中文文本的处理

- jieba  中文自然语言处理工具包

  - 分词  jieba.cut
  - 分词 得到词性  import jieba.posseg as pseg
    - pair(分词结果,词性) = pseg.cut(string_data)
  - 分词 计算权重
    - jieba.analyse.extract_tags(数据，topK=取出权重最高的前K个词，withWeight=是否显示权重，allowPOS=分词之后会保留的词性 ，withFlag=结果是否显示词性）

- wordcloud 词云

  ```python
  mask = np.array(Image.open('wordcloud.jpg'))  # 定义词频背景
  wc = wordcloud.WordCloud(
      font_path='C:/Windows/Fonts/simhei.ttf',  # 设置字体格式，不设置将无法显示中文
      mask=mask,  # 设置背景图
      max_words=200,  # 设置最大显示的词数
      max_font_size=100,  # 设置字体最大值
     
  )
  wc.generate_from_frequencies(word_counts)  # 从字典生成词云
  image_colors = wordcloud.ImageColorGenerator(mask)  # 从背景图建立颜色方案
  wc.recolor(color_func=image_colors)  # 将词云颜色设置为背景图方案
  plt.imshow(wc)  # 显示词云
  plt.axis('off')  # 关闭坐标轴
  ```

  

- 金融风控相关业务
  - 流程 
    - 数据采集 反欺诈 策略 风控模型 催收
  - 数据采集
    - 会和其它机构合作，购买数据（同盾）
    - 人民银行征信报告
  - 反欺诈 策略
    - 识别骗钱的用户
    - 主要还是策略 算法的效果不佳，样本太少，获取样本的代价太高
  - 风控模型
    - ABC评分卡
    - 贷前  申请评分卡 Application score card    A卡可以用客户历史逾期天数最大的天数
    - 贷中  行为评分卡 Behavior score card       B卡则可以多期借款中逾期最大的一次
    - 贷后   催收评分卡  Collection score card   C卡 是否被内催催回来定义y
  - 催收
    - 催收是风控的最终手段。这个环节可以产生很多对模型有帮助的数据。比如催收记录的文字描述、触达率、欺诈标签等等。

### 特征处理

- 反欺诈规则数据来源
  - 个人基本特征
  - 设备特征
  - 文本信息
  - 三方数据
- 验证是否存在一批用户在上述这些规则中有多条相似的情况，相似的规则越多 这一批用户是欺诈的可能性就越大

### 特征工程_特征选择

- Filter  过滤 过滤掉无关（和目标关系不大的）特征

  - 移除低方差的特征

    from sklearn.feature_selection import VarianceThreshold

    - VarianceThreshold(threshold=(.8 * (1 - .8)))
    - 传入的参数 方差阈值 每一列都会计算一个方差 如果方差低于这个阈值，这一列就会被删掉

  - 验证特征和目标之间是否有关联

    - 卡方检验

      ```python
      from sklearn.feature_selection import SelectKBest
      from sklearn.feature_selection import chi2
      #chi2 代表使用卡方检验 去找最合适的值
      # k = 2 找两个最合适的特征
      X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
      X_new.shape
      ```

      

    - 皮尔逊相关系数

- Wrapper  

  - RFE  递归特征消除

    - 指定一个模型，利用数据丢到模型中，找到权重最小的特征（最不重要的），每训练一轮删除一个特征，用剩下的继续训练（递归），一直到达到指定的特征个数为止
    - from sklearn.feature_selection import RFE

    ```python
    from sklearn.feature_selection import RFE
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.datasets import load_iris
    
    # 使用随机森林最为递归特征筛选的模型
    rf = RandomForestClassifier()
    iris=load_iris()
    X,y=iris.data,iris.target
    # 创建RFE对象，传入RFE过程中要使用的模型对象，
    #n_features_to_select 最终要选几个特征（剩下几个）
    rfe = RFE(estimator=rf, n_features_to_select=2)
    X_rfe = rfe.fit_transform(X,y)
    ```

- embeded

  ```python
  from sklearn.feature_selection import SelectFromModel
  from sklearn.svm import LinearSVC
  lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X,y)
  model = SelectFromModel(lsvc, prefit=True)
  X_embed = model.transform(X)
  X_embed.shape
  ```

### 特征工程

- 基础特征构造
- 数据预处理
  - 数据清洗
  - 分类
    - one-hot
  - 数值
    - 离散化 分箱       年龄
    - 归一化 标准化    销售数值
  - 样本分布是否均衡
- 特征衍生
  - 
- 特征筛选
  - filter  特征过滤
    - 低方差
    - 相关性
      - 卡方
      - 皮尔逊 
  - wrapper   RFE 把数据代入到模型中，递归去除无效特征，一次去除一个
  - embeded  嵌入到模型中，让模型直接选出有效的

### 如何选择套路

- 特征重要性（变量必须对模型有贡献，也就是说必须能对客群加以区分）
- 共线性 （逻辑回归要求变量之间线性无关）
- bivar图 （逻辑回归评分卡也希望变量呈现单调趋势）
- 稳定性（PSI） （客群在每个变量上的分布稳定，分布迁移无可避免，但不能波动太大）
- **稳定性 和单调性 结合自己的业务考虑**

### PSI

- 训练一个模型 预测每一个样本最终的bad_rate
  - 1000个样本    0.4 ~0.9   针对预测的结果 分成5份
    - 0.4~0.5     200          180
    - 0.5~0.6      200          180
    - 0.6~0.7      200          180
    - 0.7~0.8      200          180
    - 0.8~0.9     200         180
  - 1000个测试数据 代入到同一个模型

#### 特征衍生

- 利用数值型的变量，根据用户ID去分组

  - 分组之后计算每一个变量在当前用户的统计信息

    - 平均值

    - 方差/标准差

    - 最大值

    - 最小值

    - 条目数量

    - 极差（最大值-最小值）

    - 计算的时候 可以使用np.nanmin  np.nanmax 

      - np.nan*** 对缺失值不敏感

      

### 合规风控业务

坏账率不能高于5%

需要记住的

- 风控业务流程
  - 数据采集（爬虫，装了应用之后手机上的数据，买的三方数据，企业内部数据（企业内部其它应用的数据))
  -  反欺诈 主要是策略做反欺诈
  -  策略 
  -  风控模型
    - A  贷前  有可能是从合作方买的 买的数据有有效期
    - B  贷中   利用现有数据
    - C  催收  
  -  催收
- 特征工程（相关的流程）
  - 数据处理
    - 数据清洗 重复值，异常值，缺失值
    - 分类/数值
    - 目标值分类问题
      - 样本不均衡的情况
      - 过采样 欠采样
      - LR 添加权重系数
  - 特征衍生
    - 数值类型的特征
      - 根据用户分组，计算用户在当前特征下，一共有多少条数，所有数据放到一起可以算
        - 平均 方差 最大 最小 极差 条目数量
    - 分类的特征
      - 根据用户分组，计算用户在当前特征下，一共产生了不同取值的情况（有几类记录）
    - 时间序列型
      - 连续3个月有消费，连续6个月有消费
      - 连续3个月有逾期，连续6个有逾期
  - 特征筛选（特征选择）
    - filter
      - 删除低方差特征
      - 卡方检验
      - 皮尔逊相关系数
    - wrapper
      - RFE  递归剃除不重要特征
        - 把数据丢到模型中，计算特征权重，每一次去掉最不重要的一个
    - embeded
      - 把数据丢到模型里，看权重，一般用L1正则，不重要的特征直接为0 剩下的就是重要的

- IsolationForest  孤立森林
  - 记住算法的原理
    - 孤立森林算法通过对样本点的孤立来检测异常值
      - 该算法利用一种名为孤立树iTree的二叉搜索树结构来孤立样本
      - 异常值的数量较少且与大部分样本的特征不同
      - 异常值会被更早的孤立出来，也即异常值会距离![iTree](https://math.jianshu.com/math?formula=iTree)的根节点更近，而正常值则会距离根节点有更远的距离
      - 相较于LOF，K-means等传统算法，孤立森林算法对高纬数据有较好的鲁棒性。
    - 假设数据集有N条数据，构建一颗iTree时，从N条数据中均匀抽样(一般是无放回抽样)出ψ个样本出来，作为这颗树的训练样本。在样本中，随机选一个特征，并在这个特征的所有值范围内(最小值与最大值之间)随机选一个值，对样本进行二叉划分，将样本中小于该值的划分到节点的左边，大于等于该值的划分到节点的右边。由此得到一个分裂条件和左、右两边的数据集，然后分别在左右两边的数据集上重复上面的过程，直到数据集只有一条记录或者达到了树的限定高度